In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd '/content/gdrive/MyDrive/Dataset/image/flowers'
!ls -l

/content/gdrive/MyDrive/Dataset/image/flowers
total 20
drwx------ 2 root root 4096 Dec  6 06:56 daisy
drwx------ 2 root root 4096 Dec  6 06:56 dandelion
drwx------ 2 root root 4096 Dec  6 06:56 rose
drwx------ 2 root root 4096 Dec  6 06:56 sunflower
drwx------ 2 root root 4096 Dec  6 06:56 tulip


In [3]:
'''
Run under google colab with python 3.7.9 and tensorflow 2.3.0
Download data from https://www.kaggle.com/alxmamaev/flowers-recognition
'''
import cv2
import tensorflow as tf
import keras
import os
print(tf.__version__)

2.3.0


In [4]:
data_path=r'/content/gdrive/MyDrive/Dataset/image/flowers'
classes=os.listdir(data_path)
print(len(classes))

5


In [5]:
for i in classes:
    print(i,len(os.listdir('{}/{}'.format(data_path,i))))

rose 784
daisy 769
sunflower 734
tulip 984
dandelion 1055


In [6]:
training_set = tf.keras.preprocessing.image_dataset_from_directory(data_path,
                                  labels='inferred',
                                  label_mode='categorical',
                                  image_size = (224, 224),
                                  batch_size = 64,
                                  shuffle=True,
                                  seed=0,
                                  validation_split=0.2,
                                  subset='training')

validation_set = tf.keras.preprocessing.image_dataset_from_directory(data_path,
                                  labels='inferred',
                                  label_mode='categorical',
                                  image_size = (224, 224),
                                  batch_size = 64,
                                  shuffle=True,
                                  seed=0,
                                  validation_split=0.2,
                                  subset='validation')


Found 4323 files belonging to 5 classes.
Using 3459 files for training.
Found 4323 files belonging to 5 classes.
Using 864 files for validation.


In [7]:
from tensorflow import keras
from tensorflow.keras.layers import Conv2D,Flatten,Dense,Input,BatchNormalization,Add

input_shape = (224,224,3)
classes=5

def Conv_block(X,X_shortcut,feature_map,stride,name):
    for i in range(2):
        X = keras.layers.ZeroPadding2D((1, 1),name='padding'+name+str(i+1))(X)
        X = keras.layers.Conv2D(feature_map, (3, 3), strides=stride, name='Conv'+name+str(i+1))(X) 
        X = keras.layers.BatchNormalization(axis=3, name='bn_conv'+name+str(i+1))(X)
        if i == 0:
            X = keras.layers.Activation('relu',name='RELU_'+name+str(i+1))(X)
        else:
            X=keras.layers.Add(name='Skip_Connect_'+name)([X, X_shortcut])
            X = keras.layers.Activation('relu',name='RELU_'+name)(X)
            X_shortcut=X
    return X,X_shortcut

def skip_connect_block(X,X_shortcut,feature_map,name):
    X_shortcut=keras.layers.ZeroPadding2D((1, 1),name='skip_connect_padding'+name)(X_shortcut)
    X_shortcut = keras.layers.Conv2D(feature_map, (3, 3), strides=(2,2), name='skip_connect_Conv'+name)(X_shortcut)
    X_shortcut = keras.layers.BatchNormalization(axis=3, name='skip_connect_bn'+name)(X_shortcut)
    
    for i in range(2):
        X = keras.layers.ZeroPadding2D((1, 1),name='padding'+name+str(i+1))(X)
        if i==0:
            X = keras.layers.Conv2D(feature_map, (3, 3), strides=(2,2), name='Conv'+name+str(i+1))(X) 
        else:
            X = keras.layers.Conv2D(feature_map, (3, 3), strides=(1,1), name='Conv'+name+str(i+1))(X) 
            
        X = keras.layers.BatchNormalization(axis=3, name='bn_conv'+name+str(i+1))(X)
        if i == 0:
            X = keras.layers.Activation('relu',name='RELU_'+name+str(i+1))(X)
        else:
            X=keras.layers.Add(name='Skip_Connect_'+name)([X, X_shortcut])
            X = keras.layers.Activation('relu',name='RELU_'+name)(X)
            X_shortcut=X
    return X,X_shortcut
#Model
def Resnet34(input_shape,classes):
    X_input = keras.layers.Input(input_shape,name='Input')
    '''
    Stardardization of image
    '''
    X = keras.layers.experimental.preprocessing.Rescaling(1./255)(X_input)
    X = keras.layers.ZeroPadding2D((3, 3),name='padding_0')(X)
    
    #Conv1
    X = keras.layers.Conv2D(64, (7, 7), strides=(2, 2), name='Conv1')(X)
    X = keras.layers.BatchNormalization(axis=3, name='bn_conv1')(X)
    X = keras.layers.ZeroPadding2D((1, 1),name='padding_1')(X)
    X = keras.layers.Activation('relu',name='RELU_1')(X)
    X = keras.layers.MaxPooling2D((3, 3), strides=(2, 2), name='max2d_1')(X)
    
    X_shortcut=X
    
    #Conv2
    X,X_shortcut=Conv_block(X,X_shortcut,64,(1, 1),'2.1.')
    X,X_shortcut=Conv_block(X,X_shortcut,64,(1, 1),'2.2.')
    X,X_shortcut=Conv_block(X,X_shortcut,64,(1, 1),'2.3.')
    
    #Conv3
    X,X_shortcut=skip_connect_block(X,X_shortcut,128,'3.1.')
    X,X_shortcut=Conv_block(X,X_shortcut,128,(1, 1),'3.2.')
    X,X_shortcut=Conv_block(X,X_shortcut,128,(1, 1),'3.3.')
    X,X_shortcut=Conv_block(X,X_shortcut,128,(1, 1),'3.4.')
    
    #Conv4
    X,X_shortcut=skip_connect_block(X,X_shortcut,256,'4.1.')
    X,X_shortcut=Conv_block(X,X_shortcut,256,(1, 1),'4.2.')
    X,X_shortcut=Conv_block(X,X_shortcut,256,(1, 1),'4.3.')
    X,X_shortcut=Conv_block(X,X_shortcut,256,(1, 1),'4.4.')
    X,X_shortcut=Conv_block(X,X_shortcut,256,(1, 1),'4.5.')
    X,X_shortcut=Conv_block(X,X_shortcut,256,(1, 1),'4.6.')
    
    #Conv5
    X,X_shortcut=skip_connect_block(X,X_shortcut,512,'5.1.')
    X,X_shortcut=Conv_block(X,X_shortcut,512,(1, 1),'5.2.')
    X,X_shortcut=Conv_block(X,X_shortcut,512,(1, 1),'5.3.')
    
    X = keras.layers.AveragePooling2D((2,2), name="avg_pool")(X)
    #Fully connect
    X = keras.layers.Flatten(name='Flatten')(X)
    X=keras.layers.Dense(classes, activation='softmax', name='Fully_connected')(X)
    model = keras.models.Model(inputs = X_input, outputs = X, name='ResNet')
    return model

model=Resnet34(input_shape,classes)
print(model.summary())

Model: "ResNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           Input[0][0]                      
__________________________________________________________________________________________________
padding_0 (ZeroPadding2D)       (None, 230, 230, 3)  0           rescaling[0][0]                  
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 64) 9472        padding_0[0][0]                  
_____________________________________________________________________________________________

In [8]:
loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)

model.compile(optimizer = 'adam', loss = loss, metrics = ['accuracy'])
history=model.fit(training_set,epochs=50,validation_data=validation_set)

Epoch 1/50
55/55 [==============================] - 2287s 42s/step - loss: 2.4545 - accuracy: 0.3585 - val_loss: 35.1422 - val_accuracy: 0.2315
Epoch 2/50
55/55 [==============================] - 54s 985ms/step - loss: 1.3739 - accuracy: 0.4941 - val_loss: 2.6279 - val_accuracy: 0.1748
Epoch 3/50
55/55 [==============================] - 54s 987ms/step - loss: 1.3172 - accuracy: 0.5184 - val_loss: 2.5653 - val_accuracy: 0.2442
Epoch 4/50
55/55 [==============================] - 54s 986ms/step - loss: 1.4059 - accuracy: 0.5074 - val_loss: 1.9031 - val_accuracy: 0.2569
Epoch 5/50
55/55 [==============================] - 55s 994ms/step - loss: 1.3519 - accuracy: 0.5574 - val_loss: 2.2730 - val_accuracy: 0.2917
Epoch 6/50
55/55 [==============================] - 54s 987ms/step - loss: 1.1612 - accuracy: 0.5958 - val_loss: 2.2370 - val_accuracy: 0.3044
Epoch 7/50
55/55 [==============================] - 54s 988ms/step - loss: 1.2036 - accuracy: 0.5779 - val_loss: 1.7181 - val_accuracy: 0.335